# How these files are generated...

I use [readermode premium](https://readermode.io/) to covert text. it seems to do better than firefox reader mode....

Then I use this [hacked up chrome extension](https://github.com/davidbstein/chrome-page-downloader) to download everything into a json ball.

In [1]:
cd ..

/Users/stein/repos/privacy-code-tools


In [2]:
from coder.api import models
import nltk.data
import html2text
import bs4
import os
import json
from coder import settings
from coder.api import models
import datetime
import re
from collections import defaultdict
import datetime

In [3]:
DL_DIR=settings.env("DOWNLOADS_FOLDER")
TOKENIZER = nltk.data.load('tokenizers/punkt/english.pickle')
DL_DIR="/Users/stein/Downloads/PrivacyPolicyDownloads"

In [4]:
CLEANING_ARTIFACTS = [
 '  * __\n  * __\n  * __',
 '  * _search_\n  * _g_translate_',
 '  \nAdd note',
 '__ __Update __ Delete'
]

def to_coding_doc(html):
    paragraphs = _to_paragraphs(html)
    return list(map(_clean_paragraph, paragraphs))
    
def _to_paragraphs(html):
    converter = html2text.HTML2Text()
    converter.protect_links=True
    converter.single_line_break=False
    converter.inline_links=False
    converter.body_width=999999999999
    converter.strong_mark="__"
    return [p for p in converter.handle(html).split("\n\n") if p not in CLEANING_ARTIFACTS]

def _list_check(paragraph):
    return (
        paragraph.startswith(' ') 
        and 
        (
            paragraph.strip().startswith('* ') 
            or 
            paragraph.strip().startswith('1. ')
        )
    )

def _clean_paragraph(paragraph):
    if paragraph.startswith("#"):
        match = re.match(r'(^#*)(.*)', paragraph)
        level = len(match[1])
        content = match[2].strip()
        return {
            "type": "section",
            "level": level,
            "content": TOKENIZER.tokenize(content),
        }
    elif _list_check(paragraph):
        list_items = []
        for raw_list_item in paragraph.split('\n'):
            list_items.append(
                re.match(r'(?P<space>^ *?)(?P<bullet>.+?) (?P<content>.*)', raw_list_item).groupdict() 
            )
        return {
            "type": "list",
            "content": [
                {
                    "depth": len(list_item['space'])//2,
                    "bullet": list_item['bullet'],
                    "content": TOKENIZER.tokenize(list_item['content'])
                } for list_item in list_items]
        }
    else:
        return {
            "type": "text",
            "content": TOKENIZER.tokenize(paragraph)
        }

In [20]:
re.match(r"\w+\.\w+", ".DS_Store")

In [21]:
# this is terrible and needs to be rewritten.
parsed_docs = defaultdict(dict)
for site_name in os.listdir(DL_DIR):
    if not re.match(r"\w+\.\w+", site_name):
        continue
    print(f"scanning: {site_name}")
    filenames = [f for f in os.listdir(f"{DL_DIR}/{site_name}") if f.startswith("pp_snapshot")]
    for fn in sorted(filenames):
        print(fn)
        with open(f'{DL_DIR}/{site_name}/{fn}') as f:
            content = json.load(f)
            parsed_docs[site_name][fn[-len("yyyy-mm-ddThh_mm_ss.mmmZ.json"):-len('.json')]] = {
                "raw_content": content,
                "parsed_content": to_coding_doc(content['cleaned_html']),
            }

scanning: xhamster.com
pp_snapshot-xhamster.com.2021-06-23T07_35_53.594Z.json
pp_snapshot-xhamster.com.2021-06-23T07_35_54.568Z.json
pp_snapshot-xhamster.com.2021-06-23T07_35_55.503Z.json
pp_snapshot-xhamster.com.2021-06-23T07_35_56.494Z.json
scanning: webmediabrands.com
pp_snapshot-www.mediabistro.com.2021-06-23T07_43_19.310Z.json
pp_snapshot-www.mediabistro.com.2021-06-23T07_43_20.534Z.json
pp_snapshot-www.mediabistro.com.2021-06-23T07_43_21.621Z.json
scanning: completegenomics.com
pp_snapshot-www.completegenomics.com.2021-06-23T07_53_56.226Z.json
pp_snapshot-www.completegenomics.com.2021-06-23T07_53_57.652Z.json
pp_snapshot-www.completegenomics.com.2021-06-23T07_53_58.991Z.json
scanning: twitter.com
pp_snapshot-blog.twitter.com.2021-06-23T07_48_28.588Z.json
pp_snapshot-gdpr.twitter.com.2021-06-23T07_48_25.998Z.json
pp_snapshot-privacy.twitter.com.2021-06-23T07_48_25.025Z.json
pp_snapshot-privacy.twitter.com.2021-06-23T07_48_26.935Z.json
pp_snapshot-privacy.twitter.com.2021-06-23T07_

In [11]:
def create_policy_instance_content(date_to_content_list):
    """
    takes the parsed content of the JSON from the scape and turns it into the content object for the document scanner
    """
    to_ret_parsed = []
    to_ret_raw = []
    for i, (fname, content) in enumerate(sorted(date_to_content_list)):
        parsed_content = content['parsed_content']
        raw_content = content['raw_content']
        ordinal = chr(i+ord("A"))
        to_ret_parsed.append({
            "title": f'DOCUMENT {ordinal} - {parsed_content[0]["content"][0]}',
            "ordinal": ordinal,
            "content": parsed_content,
        })
        to_ret_raw.append(raw_content)
    return to_ret_parsed, to_ret_raw


In [29]:
for site_name in parsed_docs.keys():
    print(site_name)
    policy = models.Policy.objects.get(site_name=site_name)
    assert policy

for site_name in parsed_docs.keys():
    print(site_name)
    policy = models.Policy.objects.get(site_name=site_name)
    pi_content, raw_pi_content = create_policy_instance_content(parsed_docs[site_name].items())
    scan_dt = datetime.datetime.now()

    raw_policy_instance = models.RawPolicyInstance(
        policy_id=policy.id,
        raw_content_blocks=raw_pi_content,
        capture_date=scan_dt,
        capture_source="chrome-addon"
    )
    raw_policy_instance.save()
    policy_instance = models.PolicyInstance(
        policy_id=policy.id,
        raw_policy_id=raw_policy_instance.id,
        scan_dt=raw_policy_instance.capture_date,
        content=pi_content
    )
    policy_instance.save()
    policy.scan_count+=1
    policy.save()
    print(policy.company_name)
    print('\n'.join([d['title'] for d in policy_instance.content]))
    print("\n")

xhamster.com
webmediabrands.com
completegenomics.com
twitter.com
linkedin.com
pinterest.com
chaturbate.com
facebook.com
pornhub.com
xvideos.com
livejasmin.com
dpreview.com
sediabio.com
xhamster.com
xHamster's Free Porn Videos
DOCUMENT A - Terms & conditions / User agreement
DOCUMENT B - Privacy policy | xHamster
DOCUMENT C - DMCA | xHamster
DOCUMENT D - Parental Controls | xHamster


webmediabrands.com
webmediabrands.com/
DOCUMENT A - Privacy Policy | Mediabistro
DOCUMENT B - Terms of Use | Mediabistro
DOCUMENT C - Terms of Service Agreement | Mediabistro


completegenomics.com
completegenomics.com/
DOCUMENT A - Terms and Conditions - Complete Genomics
DOCUMENT B - Privacy Policy - Complete Genomics
DOCUMENT C - Public Genome Data - Complete Genomics


twitter.com
Twitter
DOCUMENT A - Twitter Terms of Service
DOCUMENT B - Privacy Policy
DOCUMENT C - Personalization and data / Twitter
DOCUMENT D - Subprocessors
DOCUMENT E - GDPR
DOCUMENT F - Twitter CCPA FAQ
DOCUMENT G - Global DPA
DOCU

In [33]:
for site_name in parsed_docs.keys():
    print(site_name)
    policy = models.Policy.objects.get(site_name=site_name)
    assert policy
    policy.progress['loaded'] = {"status": "done"}
    policy.save()

xhamster.com
webmediabrands.com
completegenomics.com
twitter.com
linkedin.com
pinterest.com
chaturbate.com
facebook.com
pornhub.com
xvideos.com
livejasmin.com
dpreview.com
sediabio.com


In [3]:
coder_emails = ['gfc9001@nyu.edu', 'kat9234@nyu.edu', 'mc8562@nyu.edu']
policy_instances = [3, 5,11,10,12,2]
to_exclude = 0
for policy_instance_id in policy_instances:
    to_exclude = (to_exclude + 1) % 3
    pi = models.PolicyInstance.objects.get(id=policy_instance_id)
    p = models.Policy.objects.get(id=pi.policy_id)
    for i, coder_email in enumerate(coder_emails):
        if i == to_exclude:
            continue
        a = models.Assignment(
            url=f"/c/2021_PP/code-policy/{pi.id}-{p.site_name}/6/",
            coder_email=coder_email,
            status="ASSIGNED",
            type=1,
            label=f"Code Policy: {p.company_name}"
        )
        a.save()

In [51]:
p.__dict__

{'_state': <django.db.models.base.ModelState at 0x12b37d7c0>,
 'id': 241,
 'project': 1,
 'company_name': 'Twitter',
 'site_name': 'twitter.com',
 'locale': 'en',
 'alexa_rank': 39,
 'alexa_rank_US': 31,
 'urls': {},
 'start_date': datetime.date(2000, 1, 21),
 'end_date': None,
 'last_scan_dt': None,
 'scan_count': 1,
 'categories': ['PP_V7'],
 'meta': {'request': {'arguments': {'argument': {'name': 'responsegroup',
     'value': 'Rank,RankByCountry,UsageStats,AdultContent,Speed,Language,OwnedDomains,SiteData'}}},
  'contentdata': {'speed': {'percentile': '81', 'medianloadtime': '873'},
   'dataurl': 'twitter.com',
   'language': {'locale': 'en'},
   'sitedata': {'title': 'Twitter',
    'description': 'Social networking and microblogging service utilising instant messaging, SMS or a web interface.',
    'onlinesince': '21-Jan-2000'},
   'adultcontent': 'no',
   'owneddomains': {}},
  'trafficdata': {'rank': '39',
   'dataurl': 'twitter.com',
   'rankbycountry': {'O': {'rank': {},
     

In [53]:
pi = models.PolicyInstance.objects.get(id=5)
p = models.Policy.objects.get(id=pi.policy_id)
print(f"{p.company_name}..")

Twitter..5-twitter.com


In [ ]:
'194-xvideos.com', '241-twitter.com', '118-pornhub.com', '131-livejasmin.com', '142-xhamster.com', '237-webmediabrands.com'